<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer_train/notebooks/label_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [ ]:
! pip install datasets

In [68]:
from transformers import pipeline
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

In [6]:
classifier = pipeline("zero-shot-classification", device = 0, model = 'valhalla/distilbart-mnli-12-1')

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
dataset = 'cnn_dailymail'
version = '3.0.0'

In [7]:
from datasets import load_dataset

dataset = load_dataset(dataset, version)

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
# dataset_subset = dataset['test'].shard(num_shards=100, index=0)

In [29]:
# output = classifier(dataset_subset['article'], candidate_labels)

In [49]:
# subset_indices = []

In [50]:
# for i, article in enumerate(output):
#   if article['labels'][0] == 'about sports' and article['scores'][0]>0.6:
#     subset_indices.append(i)

In [51]:
# subset_indices

[14, 15, 16, 22, 30, 33, 40, 43]

In [41]:
# a = dataset_subset.select(subset_indices)

In [42]:
a.save_to_disk('sports_test.hf')

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

In [68]:
output[0].keys()

dict_keys(['sequence', 'labels', 'scores'])

In [ ]:
def process_and_save():
  pass

In [66]:
def classify_and_subset(dataset, column, classifier, label, threshold):
  '''
  classifies a text into being about the label or not about the label. 
  text is contained in specified column of dataset. 
  threshold is level of certainty required (0-1)
  returns a subsetted dataset
  '''
  subset_indices = []
  # Zero-shot classification requires all the possible labels listed out. 
  labels = [f'about {label}', f'not about {label}']
  for i, article in tqdm(enumerate(classifier(KeyDataset(dataset, column), labels))):
    if article['labels'][0] == labels[0] and article['scores'][0]>threshold:
      subset_indices.append(i)
  return dataset.select(subset_indices)

In [69]:
subset = classify_and_subset(dataset_subset, 'article', classifier, 'sports', 0.6)

115it [00:26,  4.30it/s]


In [71]:
train_data_new = classify_and_subset(dataset['test'], 'article', classifier, 'sports', 0.6)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
11490it [42:18,  4.53it/s]


In [73]:
train_data_new.save_to_disk('cnn_test_sports.hf')

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]

In [74]:
client = storage.Client()

In [78]:
train_data_new

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1020
})

In [79]:
from datasets import load_from_disk
dataset2 = load_from_disk('cnn_test_sports.hf')

In [80]:
dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1020
})

In [82]:
dataset3 = load_from_disk('cnn_test_sports_downloaded.hf')

In [83]:
dataset3

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1020
})

In [84]:
dataset3[4]

{'article': '(CNN)Hockey player Jarret Stoll of the L.A. Kings was arrested Friday at the swimming pool of a Las Vegas resort on a drug-possession charge, CNN affiliate KSNV reported, citing a police spokesman. Stoll, 32, was charged with possession of controlled substances, including cocaine and ecstasy, according to KSNV. He was released from the Clark County Detention Center late Friday on $5,000 bail. The Kings said in a statement, "We are aware of police reports out of Clark County, Nevada regarding Jarret Stoll. Our organization is concerned and has begun conducting a thorough internal investigation. While we continue to actively gather facts, we are withholding further comment at this time." The Canadian player is a center and has been with the Kings since 2008. The Kings, who won the Stanley Cup two of the past three seasons, did not make the NHL playoffs this season. He is reportedly involved with TV personality Erin Andrews, who is a Fox Sports reporter and co-hosts "Dancing 

Trying cloud - decided against it due to credential issues from colab

In [75]:
bucket = client.bucket('news_data_subset')

In [76]:
blob=bucket.blob('sports_test.hf/dataset.arrow')

In [77]:
blob.upload_from_filename('sports_test.hf/dataset.arrow')

RefreshError: ignored

In [72]:
from google.cloud import storage

In [ ]:
def save_to_cloud(dataset, bucket_name, machine_filename, cloud_filename):
  # dataset.save_to_disk(machine_filename)
  client = storage.Client()
  bucket = client.bucket(bucket_name)
  for i, filename in enumerate(images.keys()):
    blob = bucket.blob(cloud_filename)
    blob.upload_from_filename(machine_filename)
    # # Delete local data after uploading to cloud
    # if os.path.exists(machine_filename):
    #     os.remove(machine_filename)

In [ ]:
save_to_cloud('news_data_subset', 'sports_test.hf', 'sports_test.hf')

In [ ]:
save_to_cloud('news_data_subset', 'cnn_test_sports.hf', 'cnn_test_sports.hf')